# Translated Periodicity
This is when you want to transfer information from one side of the block to the other. Example would be top to bottom or left to right, etc. Each colored cube represents a block. From previous tutorials, a block consists of a set of vertices.

![cube wireframe](https://github.com/nasa/Plot3D_utilities/blob/main/colab/images/block_wireframe.png?raw=true)




## Steps Overview
1. Find Connectivity - This splits the faces of all the blocks so that they match
2. Build the Block connectivity matrix using connectivity - Uses the split faces to determine which blocks are connected. value of 1 = connected, 0 = not searched yet, -1 = not connected
3. Search for connected faces on the z and y axis - Uses the block connectivity to find the outer connected faces. So, all the top faces for instance.
4. Find translated periodicity - Uses the connected faces e.g. the left and right, or top and bottom, or front and back. Checks the connected faces to see if any of them match faces from the other side.

# Environment Setup
This step is relatively short. Run the code below to install plot3d

In [1]:
!pip install plot3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.0/274.0 kB 6.9 MB/s eta 0:00:00


## Lets copy some files from the GitHub over
This file is used an example of how to export the connectivity to a format where GlennHT can read.


In [2]:
# Clone the source code for GlennOPT
!git clone https://github.com/nasa/plot3d_utilities.git

Cloning into 'plot3d_utilities'...
remote: Enumerating objects: 3141, done.
remote: Counting objects: 100% (423/423), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 3141 (delta 333), reused 305 (delta 266), pack-reused 2718 (from 2)
Receiving objects: 100% (3141/3141), 130.50 MiB | 19.30 MiB/s, done.
Resolving deltas: 100% (2148/2148), done.


In [3]:
# Copy the file we need
!cp plot3d_utilities/python/examples/block_test/glennht_con.py .

## GlennHT Connectivity Format
The glennht connectivity file is organized in 2 sections. The first section contains all the matching faces. The first number is the number of face pairs.

![Glennht connectivity format](https://github.com/nasa/Plot3D_utilities/blob/main/colab/images/glennht_connectivity.png?raw=true)

The second section show below shows the nonconnected faces also known as `outer_faces` in the code.

![Glennht connectivity format](https://github.com/nasa/Plot3D_utilities/blob/main/colab/images/glennht_connectivity2.png?raw=true)

## Import some headers
The code below are the functions from plot3d whcih will be used to find connectivity and periodicity.

In [4]:
import sys, os, pickle
import numpy as np
from plot3d import read_plot3D, connectivity_fast,translational_periodicity, block_connection_matrix, find_bounding_faces
from plot3d import outer_face_dict_to_list, match_faces_dict_to_list

In [5]:
#%% Find connectivity
def dump_data(data):
    with open('block_data.pickle','wb') as f:
        pickle.dump(data,f)

def read_data():
    with open('block_data.pickle','rb') as f:
        return pickle.load(f)

## Download the Geometry
Run the code below to download the geometry into your colab environment

In [6]:
!wget https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/iso65_64blocks.xyz

--2025-08-27 14:12:48--  https://nasa-public-data.s3.amazonaws.com/plot3d_utilities/iso65_64blocks.xyz
Resolving nasa-public-data.s3.amazonaws.com (nasa-public-data.s3.amazonaws.com)... 52.217.230.57, 3.5.27.221, 52.217.105.76, ...
Connecting to nasa-public-data.s3.amazonaws.com (nasa-public-data.s3.amazonaws.com)|52.217.230.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3773956 (3.6M) [binary/octet-stream]
Saving to: ‘iso65_64blocks.xyz’

iso65_64blocks.xyz  100%[===================>]   3.60M  --.-KB/s    in 0.1s    

2025-08-27 14:12:48 (27.2 MB/s) - ‘iso65_64blocks.xyz’ saved [3773956/3773956]



# Importing the mesh

In [7]:
blocks = read_plot3D('iso65_64blocks.xyz',binary=True,read_double=False)

## Find the Connectivity

In [8]:
print('Finding connectivity')
face_matches, outer_faces = connectivity_fast(blocks)
[m.pop('match',None) for m in face_matches] # Remove the dataframe
print('Organizing split and outerfaces')
all_faces = match_faces_dict_to_list(blocks,face_matches)
all_faces.extend(outer_face_dict_to_list(blocks,outer_faces))
all_faces = [m.to_dict() for m in all_faces]
data = {
            "face_matches":face_matches,
            "outer_faces":outer_faces,
            "all_faces":all_faces
        }
dump_data(data)

Finding connectivity
gcd to use 16


Checking connections block 63 with 60: 100%|██████████| 384/384 [00:11<00:00, 34.80it/s]


Organizing split and outerfaces


## Create the Block to Block connection matrix
It's important to find the connectivity first because it splits up all the partial matching faces. Those partial matching faces are then used to build a block to block connection matrix. This matrix is essential because it ultimately speeds up finding connected faces - useful for defining boundary conditions.

In [9]:
print('Creating block connection matrix')
c = block_connection_matrix(blocks,all_faces)
data["connectivity_matrix"]=c
dump_data(data)

Creating block connection matrix


Building connectivity: checking block 0:   0%|          | 6/2016 [00:00<00:44, 44.99it/s]

[nodes] blocks 0 and 1 connected via blk: 0 [0,0,1,1,1,1] <-> blk: 1 [0,0,0,1,1,0]
[nodes] blocks 0 and 2 connected via blk: 0 [0,1,0,1,1,1] <-> blk: 2 [0,0,0,1,0,1]
[nodes] blocks 0 and 4 connected via blk: 0 [1,0,0,1,1,1] <-> blk: 4 [0,0,0,0,1,1]


Building connectivity: checking block 1:   4%|▎         | 71/2016 [00:02<00:58, 33.46it/s]

[nodes] blocks 1 and 3 connected via blk: 1 [0,1,0,1,1,1] <-> blk: 3 [0,0,0,1,0,1]
[nodes] blocks 1 and 5 connected via blk: 1 [1,0,0,1,1,1] <-> blk: 5 [0,0,0,0,1,1]
[nodes] blocks 1 and 8 connected via blk: 1 [0,0,1,1,1,1] <-> blk: 8 [0,0,0,1,1,0]


...

[nodes] blocks 57 and 59 connected via blk: 57 [0,1,0,1,1,1] <-> blk: 59 [0,0,0,1,0,1]
[nodes] blocks 57 and 61 connected via blk: 57 [1,0,0,1,1,1] <-> blk: 61 [0,0,0,0,1,1]
[nodes] blocks 58 and 59 connected via blk: 58 [0,0,1,1,1,1] <-> blk: 59 [0,0,0,1,1,0]


Building connectivity: checking block 60: 100%|█████████▉| 2011/2016 [01:09<00:00, 32.40it/s]

[nodes] blocks 58 and 62 connected via blk: 58 [1,0,0,1,1,1] <-> blk: 62 [0,0,0,0,1,1]
[nodes] blocks 59 and 63 connected via blk: 59 [1,0,0,1,1,1] <-> blk: 63 [0,0,0,0,1,1]
[nodes] blocks 60 and 61 connected via blk: 60 [0,0,1,1,1,1] <-> blk: 61 [0,0,0,1,1,0]


Building connectivity: checking block 62: 100%|██████████| 2016/2016 [01:10<00:00, 28.78it/s]

[nodes] blocks 60 and 62 connected via blk: 60 [0,1,0,1,1,1] <-> blk: 62 [0,0,0,1,0,1]
[nodes] blocks 61 and 63 connected via blk: 61 [0,1,0,1,1,1] <-> blk: 63 [0,0,0,1,0,1]
[nodes] blocks 62 and 63 connected via blk: 62 [0,0,1,1,1,1] <-> blk: 63 [0,0,0,1,1,0]


## Find bounding faces
This finds for example all the faces on the right side of the block. Bounded axes are passed as "x","y",or "z".

![bounded faces](https://github.com/nasa/Plot3D_utilities/blob/main/colab/images/block_bounded_faces.png?raw=true)

## Find Periodicity
Now that we have the bounded faces, we can search for peridocity. In this example, we find periodicity in all directions: x, y, and z.

In [17]:
data = read_data()
all_faces = data['all_faces']
connectivity_matrix = data['connectivity_matrix']

#%% Find bounding Faces
z_periodic_faces_export, periodic_faces, outer_faces = translational_periodicity(blocks,all_faces,translational_direction='z')
x_periodic_faces_export, periodic_faces, outer_faces = translational_periodicity(blocks,outer_faces,translational_direction='x')
y_periodic_faces_export, periodic_faces, outer_faces = translational_periodicity(blocks,outer_faces,translational_direction='y')
data['x_periodic'] = x_periodic_faces_export
data['z_periodic'] = z_periodic_faces_export
data['y_periodic'] = y_periodic_faces_export
data['outer_faces'] = outer_faces
dump_data(data)

## Exporting to GlennHT Format
Because, why not.

In [18]:
data = read_data()
matched_faces = data['x_periodic']
matched_faces.extend(data['y_periodic'])
matched_faces.extend(data['z_periodic'])
matched_faces.extend(data['face_matches'])

# in this case we only have matched faces and no outer faces but lets filter it anyways
outer_faces = data['outer_faces']
outer_faces = outer_face_dict_to_list(blocks,outer_faces)
matched_faces = list(set(match_faces_dict_to_list(blocks,matched_faces)))
outer_faces = [o for o in outer_faces if o not in matched_faces]

print(f"Number of outer_faces: {len(outer_faces)}") # this should be 0

Number of outer_faces: 0


In [19]:
outer_faces

[]